## 概率图模型

### 模型起源

我们想对联合概率分布 $P(Y)$ 建模

问题是，$Y$的维度很高时，eg. K维，<u>需要估计的参数量是指数级的</u>：假设$Y$的每个分量都是离散型随机变量且只有2种取值，那么参数个数为 $2^K - 1$ （设想离散随机变量的概率表）

于是有了<u>条件独立性假设</u>

我们用**图**这种数据结构来表示条件独立性

- 有向图，即**贝叶斯网络**：：弧表示因果关系
- 无向图，即**马尔科夫随机场**：边表示概率依赖关系，但不一定是因果关系

### 条件独立性的几种情况（图）

1. X -> Z -> Y  间接因果关系 已知Z时X独立于Y
2. Y -> X1,X2     共因关系   X1 X2 不独立，但已知Y时X1X2独立  （NaiveBayes的假设）
3. X1,X2 -> Y     共果关系   X1 X2 独立，但已知Y时X1X2不独立

情况2 由表象特征“推断”本质原因   
情况3 由影响因素“预测”结果

### 数学定义

#### 有向图

*local conditional probability distribution，局部条件概率分布*： 已知其父节点情况下，该节点的条件概率分布

$$ P(Y_k | Y_{Father(k)}) $$

*Bayes Network，贝叶斯网络*：  联合概率分布可以分解为所有分量局部条件概率分布的乘积

$$ P(Y) = \prod_{k=1}^K P(Y_k | Y_{Father(k)}) $$

#### 无向图

*local Markov property, 局部马尔科夫性质*  给定邻居的情况下独立于所有其他变量

$$ X_k \perp X_{/k /N(k)} | X_{N(k)}$$

ie. 
$$ P(X_k | X_{/k}) = P(X_k | X_{N(k)}) $$

*Markov Random Field, 马尔科夫随机场*： 所有分量都满足局部马尔科夫性质

### 回到主线：如何利用条件独立性分解联合概率分布？

有向图：拓扑排序，链式法则。

接下来重点讨论无向图。

首先定义 *团(clique)* 和 *最大团(maxium clique)* ：两两有边。

概率无向图的*因子分解，factorization* : 将 联合概率分布 分解为 定义在最大团上的非负函数的乘积。

怎么确定能分解出来呢？Hammersley-Clifford定理！

> Hammersley-Clifford 定理
> 
> 满足局部马尔科夫性质 <=> 可分解
> 
> ie.
> $$ P(X) \overset{\text{can be written as }}{=} \frac{1}{Z} \prod_{c \in \mathcal{C}} \psi_c(X_c)$$

另外由于是“非负”函数，我们一般给它一个指数函数的形式

$$  \psi_c(X) = \exp (- H_c(X)) $$

then 连乘变连加

$$ P(X) = \frac{1}{Z} \exp(- \sum_{c \in \mathcal{C}} H_c(x_c)) $$

### 条件随机场、链式条件随机场、以及非负函数 $\psi(\cdot)$ 的一种具体形式

#### 条件随机场

X,Y两种变量，X可观测，Y不可观测。若已知X条件下Y是一个马尔科夫随机场，则$Y|X$是条件随机场。注意条件随机场是对$P(Y|X)$建模。

$$ \text{Markov Random Field, MRF} \xrightarrow[]{Y | X} \text{Conditional Random Field, CRF} $$

#### 链式条件随机场及其应用：词性标注

链式条件随机场（图）

$$ P(Y_t | Y_{/t}, X) = P(Y_t | Y_{t-1}, Y_{t+1}, X)$$

#### 非负函数$\psi(\cdot)$的一种常见具体形式：$H(\cdot)$是局部特征函数$f$的线性组合

我们设定局部条件概率的方式类似于“打分”。根据领域知识写出一系列我们认为正常情况下应该满足的“特征”或“规则”，并将每个特征赋予一个权重。局部条件概率为样本在这些特征上打分的加和，即满足的特征越多，给予的概率越高。

局部条件概率（非规范化）为局部特征函数的线性组合：

$$ \tilde P(Y_t | Y_{t-1}, X) =  \exp \Big( \sum_{k=1}^K w_k f_k(Y_{t-1}, Y_t, X, t) \Big)$$

联合概率分布

$$ P(Y|X) = \frac{1}{Z} \exp \Big(   \sum_{t=1}^T \sum_{k=1}^K w_k f_k(Y_{t-1}, Y_t, X, t)   \Big) $$

##### 简化形式：全局特征函数

根据时间 $t$ 加和，得到全局特征函数 $F(\cdot)$

$$ F(Y, X) \overset{def}{=} \sum_{t=1}^T f(Y_{t-1}, Y_t, X, t) $$

那么联合概率可以写为

$$ P(Y|X) = \frac{1}{Z} \exp \Big( \sum_{k=1}^K w_k F(Y, X)   \Big) $$

where Z is funciton of x

$$ Z(x) = \sum_{\mathbf{y}} \exp(\sum_k w_k F_k(\mathbf{y}, x))$$

##### 矩阵形式

定义时间 $t$ 上的转移矩阵 $M_t$ 

它的$[i,j]$元素为 “t-1时刻状态为i, t时刻状态为j”的非规范化条件概率，即 exp(总特征分)

$$ \begin{align}
M_t(x) [i, j] &\overset{def}{=} m_t(y_{t-1}, y_t | x) \\
              &= \exp \Big( TotalScore_t(y_{t-1} = i, y_t = j | x) \Big) \\ 
              &= \exp \Big( \sum_k w_k f_k(y_{t-1}=i, y_t=j, x, t) \Big)
\end{align}$$


如果将 start, stop 看做两个特殊的状态(排在第0位)，则一共有t+1的时刻。

$M_1$ 为从t=0(状态为start)到t=1时刻的转移矩阵：
$$ M_1 = \begin{bmatrix}
\pi_1 & \pi_2 & \pi_3 \\
0     & 0     & 0     \\
0     & 0     & 0     \\
\end{bmatrix}$$

$M_T$ 为从t=T到t=T+1(状态为stop)时刻的转移矩阵：
$$ M_T = \begin{bmatrix}
1     & 0     & 0     \\
1     & 0     & 0     \\
1     & 0     & 0     \\
\end{bmatrix}$$

矩阵M连乘表示经过n次连乘，从状态i到状态j的非规范化概率。那么归一化因子$Z(x)$为整个时间窗口上的矩阵M连乘的[start, stop]元

$$ Z(x) = (M_1(x) M_2(x) \dots M_T(x))_{[start, stop]} $$

而某个特定路径$\{y_1, y_2, \dots, y_T\}$的非规范化概率为

$$ \prod_{t=1}^T m_t(y_{t-1}, y_t | x) $$

因此，某个特定路径的条件概率为

$$ P( \mathbf{y} | x) = \frac{1}{Z(x)} \prod_{t=1}^T m_t(y_{t-1}, y_t | x) $$


#### 词性标注问题下的特征函数$f$

特征函数$f(\cdot)$一般细化为两种特征函数：状态特征函数$s(\cdot)$和转移特征函数$t(\cdot)$，它们都是0-1函数。

$$ \sum_k f_k(X, Y_{t-1}, Y_t, t) = \sum_k \lambda_k s_k (Y_t, X, t) + \sum_l \mu_l t_l (Y_{t-1}, Y_t, X, t) $$


特征函数是人为规定的，代表我们的领域知识。特征函数的形式为：满足某种规则的，函数值为1，否则为0，这里的规则就是领域知识的数学表达。


- “介词+名词是合理的结构” $t_1 = I(Y_{t-1}=P, Y_t = N)$， 权重 $\lambda_1$ = 0.7
- “以ly结尾的大概率是副词” $s_2 = I(End(Y_t)=ly, Y_t = A)$， 权重 $\mu_2$ = 0.9
- ... 